In [33]:
import numpy as np
import scipy as sp
import pyscf

from pyscf import gto, scf

In [34]:
mol = gto.Mole()
mol.atom = 'Ti 0.0 0.0 0.0; H 0.758602 0.0 0.504284; H 0.758602 0.0 -0.504284'
mol.basis = 'sto-3g'
mol.cart = True
# mol.verbose = 4
mol.build()

Helgaker (9.1.6) for number of Cartesian GTOs in a shell of total $l$.

$$N_l^c = \frac{(l+1)(l+2)}{2}$$

In [35]:
import itertools

def generate_cartesian_exponents(l):
    return [
        (item.count('x'), item.count('y'), item.count('z'))
        for item in itertools.combinations_with_replacement(['x', 'y', 'z'], l)
    ]


def generate_orbitals(mol):
    atomlist = mol._atom
    basislist = mol._basis
    orbital_description = {}
    i = 0

    for atom_index, atom in enumerate(atomlist):  # <-- Get atom index here
        atom_name = atom[0]
        atom_coords = atom[1]
        basis = basislist[atom_name]

        for shell in basis:
            l = shell[0]
            coeffs = shell[1:]
            exponents = generate_cartesian_exponents(l)

            for nx, ny, nz in exponents:
                orbital_description[i] = {
                    "Atom": atom_name,
                    "Atom Index": atom_index,  # <-- Store index here
                    "Coordinates": atom_coords,
                    "Angular Momentum": l,
                    "Exponents": (nx, ny, nz),
                    "Coefficients": coeffs
                }
                i += 1

    return orbital_description, i  # i = total orbitals


mol_orbitals, dim = generate_orbitals(mol)

In [36]:
import scipy as sp
import math


from scipy.special import factorial2

def safe_factorial2(n):
    if n <= 0:
        return 1  # define (-1)!! = 1, (0)!! = 1
    return factorial2(n)

def normalize(alpha, l, m, n):
    norm = ((2 * alpha / np.pi)**(3/2) *
            (4 * alpha)**(l + m + n) /
            (safe_factorial2(2 * l - 1) *
             safe_factorial2(2 * m - 1) *
             safe_factorial2(2 * n - 1)))**0.5
    return norm

# normalize works correctly; verified via Desmos.




In [ ]:
normalize(1, 2,0,0)

1.6459227806494894

In [47]:
normalize(1, 1, 1, 0)

2.8508218814199604

In [ ]:
# def generateEs(X_PA, X_PB, K00, p, max_l = 2):
    
#     E_tij = {}
    
#     for t in range(2*max_l + 1):
#         E_tij[t] = {}

#     E_tij[0][(0, 0)] = K00

#     def get_etij(t, i, j):
#         if t < 0 or t > i + j or i < 0 or j < 0:
#             return 0.0
#         else:
#             return E_tij[t][(i,j)]
    
#     def set_etij(t, i, j, value):
#         if t not in E_tij:
#             E_tij[t] = {}
#         if t < 0 or t > i + j or i < 0 or j < 0:
#             return
#         E_tij[t][(i,j)] = value
    
#     # sweep by total "angmom".
#     for L in range(2 * max_l + 1):
#         total_angular_momentum = L

#         # need possible combinations of (i, j) such that i + j = L
#         for i in range(total_angular_momentum + 1):
#             j = total_angular_momentum - i

#             for t in range(0, total_angular_momentum + 1):
#                 if (i,j) not in E_tij[t]:
#                     if i > j:
#                         thisval = (1 / (2 * p)) * get_etij(t - 1, i - 1, j) + X_PA * get_etij(t, i-1, j) + (t+1)* get_etij(t+1, i-1, j)
#                         set_etij(t, i, j, thisval)
#                     else:
#                         thisval = (1 / (2 * p)) * get_etij(t - 1, i, j - 1) + X_PB * get_etij(t, i, j-1) + (t+1)* get_etij(t+1, i, j-1)
#                         set_etij(t, i, j, thisval)

#     return E_tij

In [37]:
def getEtij(X_PA, X_PB, K00, p, t, i, j):

    if t < 0 or t > i + j or i < 0 or j < 0:
        return 0.0
    elif i == 0 and j == 0:
        return K00 if t == 0 else 0.0
    
    if i > j:
        return (
            (1/(2*p)) * getEtij(X_PA, X_PB, K00, p, t - 1, i - 1, j) +
            X_PA * getEtij(X_PA, X_PB, K00, p, t, i - 1, j) +
            (t + 1) * getEtij(X_PA, X_PB, K00, p, t + 1, i - 1, j)
        )
    
    else:
        return (
            (1/(2*p)) * getEtij(X_PA, X_PB, K00, p, t - 1, i, j - 1) +
            X_PB * getEtij(X_PA, X_PB, K00, p, t, i, j - 1) +
            (t + 1) * getEtij(X_PA, X_PB, K00, p, t + 1, i, j - 1)
        )

In [ ]:
# def generateEs(X_PA, X_PB, K00, p, max_l=2):
#     memo = {}
    
#     def E(t, i, j):
#         key = (t, i, j)
#         if key in memo:
#             return memo[key]
        
#         if t < 0 or t > (i + j) or i < 0 or j < 0:
#             return 0.0
#         if i == 0 and j == 0:
#             return K00 if t == 0 else 0.0
        
#         if i > j:
#             val = (1 / (2 * p)) * E(t - 1, i - 1, j) + X_PA * E(t, i - 1, j) + (t + 1) * E(t + 1, i - 1, j)
#         else:
#             val = (1 / (2 * p)) * E(t - 1, i, j - 1) + X_PB * E(t, i, j - 1) + (t + 1) * E(t + 1, i, j - 1)
        
#         memo[key] = val
#         return val
    
#     E_tij = {}
#     for L in range(2 * max_l + 1):
#         for i in range(L + 1):
#             j = L - i
#             for t in range(0, L + 1):
#                 E_tij[(t, i, j)] = E(t, i, j)
    
#     return E_tij

In [38]:
def Sprimitive(exponentsA, exponentsB, coordsA, coordsB, alphaA, alphaB):
    p = alphaA + alphaB
    mu = (alphaA * alphaB) / p
    coords_AB = [coordsA[i] - coordsB[i] for i in range(3)]
    Pcoord = [(alphaA * coordsA[i] + alphaB * coordsB[i]) / p for i in range(3)]
    rPA = [(Pcoord[i] - coordsA[i]) for i in range(3)]
    rPB = [(Pcoord[i] - coordsB[i]) for i in range(3)]

    K00x = np.exp(-mu * coords_AB[0]**2)
    K00y = np.exp(-mu * coords_AB[1]**2)
    K00z = np.exp(-mu * coords_AB[2]**2)

    # Etij_x = generateEs(rPA[0], rPB[0], K00x, p, 2)
    # Etij_y = generateEs(rPA[1], rPB[1], K00y, p, 2)
    # Etij_z = generateEs(rPA[2], rPB[2], K00z, p, 2)

    # return Etij_x[0][(exponentsA[0], exponentsB[0])] * \
    #            Etij_y[0][(exponentsA[1], exponentsB[1])] * \
    #            Etij_z[0][(exponentsA[2], exponentsB[2])] * \
    #            (np.pi/p)**(3/2)

    return getEtij(rPA[0], rPB[0], K00x, p, 0, exponentsA[0], exponentsB[0]) * \
            getEtij(rPA[1], rPB[1], K00y, p, 0, exponentsA[1], exponentsB[1]) * \
            getEtij(rPA[2], rPB[2], K00z, p, 0, exponentsA[2], exponentsB[2]) * \
            (np.pi/p)**(3/2)

In [8]:
exponentsA = (0,1,0)
exponentsB = (0,1,0)
coordsA = (0, 0, 0)
coordsB = (0, 0, 0)

alphaA = 1
alphaB = 1

Sprimitive(exponentsA, exponentsB, coordsA, coordsB, alphaA, alphaB)

0.4921753108038256

In [39]:
def generate_overlap(mol):
    orbital_description, mat_dim = generate_orbitals(mol)

    Sovlp = np.zeros((mat_dim, mat_dim))

    for i in range(mat_dim):
        for j in range(mat_dim):
            Sovlpij = 0

            orbi = orbital_description[i]
            orbj = orbital_description[j]

            for ii in range(len(orbi["Coefficients"])):
                for jj in range(len(orbj["Coefficients"])):
                    exponentsA = orbi["Exponents"]
                    exponentsB = orbj["Exponents"]
                    coordsA = orbi["Coordinates"]
                    coordsB = orbj["Coordinates"]
                    alphaA = orbi["Coefficients"][ii][0]
                    alphaB = orbj["Coefficients"][jj][0]
                    dA = orbi["Coefficients"][ii][1]
                    dB = orbj["Coefficients"][jj][1]
                    normA = normalize(alphaA, exponentsA[0], exponentsA[1], exponentsA[2])
                    normB = normalize(alphaB, exponentsB[0], exponentsB[1], exponentsB[2])

                    Sij = Sprimitive(exponentsA, exponentsB, coordsA, coordsB, alphaA, alphaB)
                    Sovlpij += dA * dB * normA * normB * Sij
            
            Sovlp[i, j] = Sovlpij
            # Sovlp[j, i] = Sovlpij
    
    return Sovlp

In [21]:
# def mod_generate_overlap(mol):
#     # for testing purposes; do we normalize d orbitals?
#     orbital_description, mat_dim = generate_orbitals(mol)

#     Sovlp = np.zeros((mat_dim, mat_dim))

#     for i in range(mat_dim):
#         for j in range(mat_dim):
#             Sovlpij = 0

#             orbi = orbital_description[i]
#             orbj = orbital_description[j]

#             for ii in range(len(orbi["Coefficients"])):
#                 for jj in range(len(orbj["Coefficients"])):
#                     exponentsA = orbi["Exponents"]
#                     exponentsB = orbj["Exponents"]
#                     coordsA = orbi["Coordinates"]
#                     coordsB = orbj["Coordinates"]
#                     alphaA = orbi["Coefficients"][ii][0]
#                     alphaB = orbj["Coefficients"][jj][0]
#                     dA = orbi["Coefficients"][ii][1]
#                     dB = orbj["Coefficients"][jj][1]

#                     if exponentsA[0] + exponentsA[1] + exponentsA[2] > 1:
#                         normA = 1
#                     else:
#                         normA = normalize(alphaA, exponentsA[0], exponentsA[1], exponentsA[2])
                    
#                     if exponentsB[0] + exponentsB[1] + exponentsB[2] > 1:
#                         normB = 1
#                     else:
#                         normB = normalize(alphaB, exponentsB[0], exponentsB[1], exponentsB[2])


#                     Sij = Sprimitive(exponentsA, exponentsB, coordsA, coordsB, alphaA, alphaB)
#                     Sovlpij += dA * dB * normA * normB * Sij
            
#             Sovlp[i, j] = Sovlpij
#             # Sovlp[j, i] = Sovlpij
    
#     return Sovlp

In [22]:
mol_orbitals, dim = generate_orbitals(mol)
mol_orbitals[0]["Exponents"]

(0, 0, 0)

In [23]:
pyscfmatrix = mol.get_ovlp()

converged SCF energy = -839.754236226295


In [24]:
mymatrix = generate_overlap(mol)

In [25]:
np.diag(pyscfmatrix)

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 2.51327412, 0.83775804,
       0.83775804, 2.51327412, 0.83775804, 2.51327412, 1.        ,
       1.        ])

Maybe I'll simply accept that PySCF and my implementation of Helgaker Chapter 9 will differ.

In [28]:
np.trace(pyscfmatrix)

25.05309649148734

In [27]:
2.51327412+2.51327412+2.51327412+0.83775804+0.83775804+0.83775804

10.053096480000002

In [26]:
np.diag(mymatrix)

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.99999999,
       0.99999999])

In [ ]:
# modmymatrix = mod_generate_overlap(mol)

In [17]:
print(np.linalg.eigvals(pyscfmatrix))
print(np.linalg.eigvals(mymatrix))

[6.45638865 1.97077332 1.40761186 0.05285528 0.05604731 1.62895423
 0.75126992 0.45931196 1.0609781  0.51156637 0.09025165 1.73475526
 1.67551608 1.52627149 1.10249886 0.97576424 0.91465143 0.50415012
 0.49796427 0.83775804 0.83775804]
[4.0770664  1.76594281 1.39773372 0.03597801 0.05350109 1.74705221
 1.06877155 1.52627149 0.75116677 0.70945534 0.45779504 0.09280827
 1.18149613 0.50737331 0.98229923 0.97576424 0.66666667 0.50489343
 0.49796427 1.         1.        ]


In [13]:
print(mymatrix)

[[ 1.00000000e+00  3.71774542e-01  2.23607807e-02 -4.12729615e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.17036544e-03  0.00000000e+00  0.00000000e+00
   1.17036544e-03  0.00000000e+00  1.17036544e-03  1.27771494e-02
   1.27771494e-02]
 [ 3.71774542e-01  1.00000000e+00  3.00383882e-01  1.36941172e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.46472919e-02  0.00000000e+00  0.00000000e+00
   5.46472919e-02  0.00000000e+00  5.46472919e-02  8.56945567e-02
   8.56945567e-02]
 [ 2.23607807e-02  3.00383882e-01  1.00000000e+00  3.95148821e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.44929947e-01  0.00000000e+00  0.00000000e+00
   5.44929947e-01  0.00000000e+00  5.4

In [14]:
print(pyscfmatrix)

[[ 1.00000000e+00  3.71774542e-01  2.23607807e-02 -4.12729615e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.85541651e-03  0.00000000e+00  0.00000000e+00
   1.85541651e-03  0.00000000e+00  1.85541651e-03  1.27771495e-02
   1.27771495e-02]
 [ 3.71774542e-01  1.00000000e+00  3.00383882e-01  1.36941171e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.66340415e-02  0.00000000e+00  0.00000000e+00
   8.66340415e-02  0.00000000e+00  8.66340415e-02  8.56945571e-02
   8.56945571e-02]
 [ 2.23607807e-02  3.00383882e-01  1.00000000e+00  3.95148821e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.63894294e-01  0.00000000e+00  0.00000000e+00
   8.63894294e-01  0.00000000e+00  8.6

In [15]:
print(modmymatrix)

[[ 1.00000000e+00  3.71774542e-01  2.23607807e-02 -4.12729615e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.15262632e-03  0.00000000e+00  0.00000000e+00
   1.15262632e-03  0.00000000e+00  1.15262632e-03  1.27771494e-02
   1.27771494e-02]
 [ 3.71774542e-01  1.00000000e+00  3.00383882e-01  1.36941172e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  6.59234073e-02  0.00000000e+00  0.00000000e+00
   6.59234073e-02  0.00000000e+00  6.59234073e-02  8.56945567e-02
   8.56945567e-02]
 [ 2.23607807e-02  3.00383882e-01  1.00000000e+00  3.95148821e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.33018671e+00  0.00000000e+00  0.00000000e+00
   1.33018671e+00  0.00000000e+00  1.3

In [51]:
orbital_description, dim = generate_orbitals(mol)
print(orbital_description)

{0: {'Atom': 'Ti', 'Atom Index': 0, 'Coordinates': [0.0, 0.0, 0.0], 'Angular Momentum': 0, 'Exponents': (0, 0, 0), 'Coefficients': [[1033.571245, 0.1543289673], [188.2662926, 0.5353281423], [50.95220601, 0.4446345422]]}, 1: {'Atom': 'Ti', 'Atom Index': 0, 'Coordinates': [0.0, 0.0, 0.0], 'Angular Momentum': 0, 'Exponents': (0, 0, 0), 'Coefficients': [[75.2512046, -0.0999672292], [17.48676162, 0.3995128261], [5.687237606, 0.7001154689]]}, 2: {'Atom': 'Ti', 'Atom Index': 0, 'Coordinates': [0.0, 0.0, 0.0], 'Angular Momentum': 0, 'Exponents': (0, 0, 0), 'Coefficients': [[5.395535474, -0.2277635023], [1.645810296, 0.2175436044], [0.635004777, 0.9166769611]]}, 3: {'Atom': 'Ti', 'Atom Index': 0, 'Coordinates': [0.0, 0.0, 0.0], 'Angular Momentum': 0, 'Exponents': (0, 0, 0), 'Coefficients': [[0.7122640246, -0.3088441215], [0.2628702203, 0.0196064117], [0.1160862609, 1.131034442]]}, 4: {'Atom': 'Ti', 'Atom Index': 0, 'Coordinates': [0.0, 0.0, 0.0], 'Angular Momentum': 1, 'Exponents': (1, 0, 0), '

In [44]:
np.allclose(3, 2.99999999)

True

In [45]:
for i in range(dim):
    for j in range(dim):
        if not np.allclose(pyscfmatrix[i, j],mymatrix[i, j]):
            print(f"Mismatch at ({i}, {j}): PySCF = {pyscfmatrix[i, j]}, My Matrix = {mymatrix[i, j]}")
            print("Orbital description: ", orbital_description[i]["Exponents"], "and", orbital_description[j]["Exponents"])

Mismatch at (0, 13): PySCF = 0.001855416513645918, My Matrix = 0.0011703654367385993
Orbital description:  (0, 0, 0) and (2, 0, 0)
Mismatch at (0, 16): PySCF = 0.001855416513645918, My Matrix = 0.0011703654367385993
Orbital description:  (0, 0, 0) and (0, 2, 0)
Mismatch at (0, 18): PySCF = 0.001855416513645918, My Matrix = 0.0011703654367385993
Orbital description:  (0, 0, 0) and (0, 0, 2)
Mismatch at (1, 13): PySCF = 0.08663404145786187, My Matrix = 0.05464729187075084
Orbital description:  (0, 0, 0) and (2, 0, 0)
Mismatch at (1, 16): PySCF = 0.08663404145786187, My Matrix = 0.05464729187075084
Orbital description:  (0, 0, 0) and (0, 2, 0)
Mismatch at (1, 18): PySCF = 0.08663404145786187, My Matrix = 0.05464729187075084
Orbital description:  (0, 0, 0) and (0, 0, 2)
Mismatch at (2, 13): PySCF = 0.8638942938156714, My Matrix = 0.5449299469745862
Orbital description:  (0, 0, 0) and (2, 0, 0)
Mismatch at (2, 16): PySCF = 0.8638942938156714, My Matrix = 0.5449299469745862
Orbital descripti

Figure this out -- why are the values different for d orbitals?

In [40]:
print(mymatrix)

[[ 1.00000000e+00  3.71774542e-01  2.23607807e-02 -4.12729615e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.17036544e-03  0.00000000e+00  0.00000000e+00
   1.17036544e-03  0.00000000e+00  1.17036544e-03  1.27771494e-02
   1.27771494e-02]
 [ 3.71774542e-01  1.00000000e+00  3.00383882e-01  1.36941172e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.46472919e-02  0.00000000e+00  0.00000000e+00
   5.46472919e-02  0.00000000e+00  5.46472919e-02  8.56945567e-02
   8.56945567e-02]
 [ 2.23607807e-02  3.00383882e-01  1.00000000e+00  3.95148821e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.44929947e-01  0.00000000e+00  0.00000000e+00
   5.44929947e-01  0.00000000e+00  5.4

In [41]:
print(pyscfmatrix)

[[ 1.00000000e+00  3.71774542e-01  2.23607807e-02 -4.12729615e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.85541651e-03  0.00000000e+00  0.00000000e+00
   1.85541651e-03  0.00000000e+00  1.85541651e-03  1.27771495e-02
   1.27771495e-02]
 [ 3.71774542e-01  1.00000000e+00  3.00383882e-01  1.36941171e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.66340415e-02  0.00000000e+00  0.00000000e+00
   8.66340415e-02  0.00000000e+00  8.66340415e-02  8.56945571e-02
   8.56945571e-02]
 [ 2.23607807e-02  3.00383882e-01  1.00000000e+00  3.95148821e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.63894294e-01  0.00000000e+00  0.00000000e+00
   8.63894294e-01  0.00000000e+00  8.6

In [15]:
pyscfeigval = np.linalg.eigvals(pyscfmatrix)
myeigval = np.linalg.eigvals(mymatrix)

In [16]:
print(pyscfeigval)
print(myeigval)

[5.49651486 2.13286349 1.66015983 1.35977781 1.30845926 0.01658315
 0.18387633 0.95049048 0.49522252 0.57710854 0.68324999 0.61310042
 1.67551608 0.98933117 1.3975684  0.61310042 0.98933117 1.3975684
 0.83775804 0.83775804 0.83775804]
[3.1311147  2.00027654 0.01285521 0.1805593  1.34995114 1.29256563
 0.49184006 0.56679544 0.9429279  0.6711655  0.69328189 1.3975684
 1.3975684  0.66666667 0.61310042 0.61310042 0.98933117 0.98933117
 1.         1.         1.        ]


[3.1311147  2.00027654 0.01285521 0.1805593  1.34995114 1.29256563
 0.49184006 0.56679544 0.9429279  0.6711655  0.69328189 1.3975684
 1.3975684  0.66666667 0.61310042 0.61310042 0.98933117 0.98933117
 1.         1.         1.        ]


In [142]:
print(ck(0, 0, 0, 0.0, 0.0))  # should be 1.0
print(ck(1, 1, 0, 0.1, 0.2))  # test p-s overlap
print(ck(2, 1, 1, 0.1, 0.2))  # test p-p overlap


1.0
1.0
1.0


In [83]:
mol_orbitals[0]

{'Atom': 'Fe',
 'Coordinates': [5.6691783736951855, 5.6691783736951855, 17.007535121085557],
 'Angular Momentum': 0,
 'Label': 0,
 'Coefficients': [[1447.400411, 0.1543289673],
  [263.6457916, 0.5353281423],
  [71.35284019, 0.4446345422]]}

In [82]:
len(mol_orbitals)

21

In [53]:
print(np.shape(mol.get_ovlp()))



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/abalaji/Aadhu-Tries-to-Code/integral_calculator/tmpdaropg9y
max_memory 4000 MB (current use 297 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -1246.37718289423
  HOMO = 0.172848707419002  LUMO = 0.319189570956139
cycle= 1 E= -1243.59940354377  delta_E= 2.78  |g|= 2.35  |ddm|= 5.89
  HOMO = -2.4186309575507  LUMO = -1.04937308815867
cycle= 2 E= -1240.49651898199  delta_E=  3.1  |g|= 2.69  |ddm|= 8.51

WARN: HOMO 0.0943540297095251 == LUMO 0.094354079218434

cycle= 3 E= -1247.31373981143  delta_E= -6.82  |g|= 0.49  |ddm|= 6.84
  HOMO = -0.532268912493751  LUMO = -0.431399056163331
cycle= 4 E= -1245.18370178767  delta_E=